In [29]:
import urllib.request
from collections import defaultdict, namedtuple
from itertools import chain
import re
import numpy as np
from heapq import heappop, heappush

# It's day 24
day = 24

# Load file from url
txt = urllib.request.urlopen(f'https://raw.githubusercontent.com/SantoSimone/Advent-of-Code/master/2016/input_files/input{day}.txt').read().decode('utf-8')
# Split lines
lines = txt.splitlines()

In [35]:
# Authors are addicted to this
# Maybe i should go for a single notebook next time

def problem_solver(start, f_heuristic, f_moves):
  # Heap with next state to be evaluated
  heap = [(f_heuristic(start), start)]
  # Dicts to keep track of previous state and cost for each state
  prev_states = {start: None}
  costs = {start: 0}

  while heap:
    (heur, state) = heappop(heap)
    if f_heuristic(state) == 0:
      # Goal state, we're done
      return step(prev_states, state)
    for new_state in f_moves(state):
      # Cost is hard-coded to 1 but we could add a cost function to signature
      new_cost = costs[state] + 1 
      if new_state not in costs or new_cost < costs[new_state]:
        # Never seen state OR better solution than previous one
        heappush(heap, (new_cost + f_heuristic(new_state), new_state))
        costs[new_state] = new_cost
        prev_states[new_state] = state

  return dict(fail=True, front=len(heap), prev=len(prev_states))

def step(prev, state):
  return ([] if state is None else step(prev, prev[state]) + [state])

def heuristic(state):
  # Calculating how many more moves we need to finish (heuristic obviously)
  return goal - len(state.collected)

def moves(state):
  # Possible moves from actual state
  x, y, collected = state
  
  for x2, y2 in neighbors(x, y):
    if legal_move(x2, y2):
      yield State(x2, y2, fs(*(chain(collected, lines[y2][x2])))) if lines[y2][x2].isdigit() and lines[y2][x2] not in collected else State(x2, y2, fs(*collected))

def neighbors(x, y):
  return (x+1, y), (x-1, y), (x, y+1), (x, y-1)

def legal_move(x, y):
  return 0 <= x < len(lines[0]) and 0 <= y < len(lines) and lines[y][x] != '#'

def obj_to_move(objs):
  for x in chain(combinations(objs, 1), combinations(objs, 2)):
    yield frozenset(x)

def get_start():
  for y, line in enumerate(lines):
    for x, ch in enumerate(line):
      if ch == '0':
        return x, y

def fs(*items): return frozenset(items)

# SETUP
State = namedtuple('State', 'x, y, collected')
goal = sum(x.isdigit() for line in lines for x in line)
start = get_start()

In [ ]:
# PART ONE
path = problem_solver(State(start[0], start[1], fs('0')), heuristic, moves)

print(f'Number of steps needed is {len(path) -1}')

In [ ]:
# PART TWO
# We need to redefine the heuristic to get into account to return to position 0
def heuristic(state):
  # Calculating how many more moves we need to finish (heuristic obviously)
  nums = goal - len(state.collected)
  dist = abs(state.x - start[0]) + abs(state.y - start[1])
  return nums + dist

path = problem_solver(State(start[0], start[1], fs('0')), heuristic, moves)

print(f'Number of steps needed is {len(path) -1}')